<a href="https://colab.research.google.com/github/zouden/mitra-test/blob/main/MitraTakeHomeR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing modification counts with DESeq2

In this exercise we will treat the counts of modifications as counts of genes, and use the differential expression package DESeq2 to compare counts between the sample groups.

The first step is to install DESeq into the Colab environment. This takes about 15 minutes.

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.13")
BiocManager::install("DESeq2")
library("DESeq2")

## Read the data file
Split the dataset into "A" and "B" counts

In [21]:
# Read the data file
countdata <- read.csv("exercise_data.csv", header=TRUE)
geneloci = countdata[c(1,2)]
counts.A = countdata[c(3,5,7,9,11,13)]
counts.B = countdata[c(4,6,8,10,12,14)]
coldata.A = data.frame(row.names = colnames(counts.A), 'group' = c('north','north','north','south','south','south'))
coldata.B = data.frame(row.names = colnames(counts.B), 'group' = c('north','north','north','south','south','south'))

,Chromosome,Position,Sample1.A,Sample1.B,Sample2.A,Sample2.B,Sample3.A,Sample3.B,Sample4.A,Sample4.B,Sample5.A,Sample5.B,Sample6.A,Sample6.B
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,chra,6,0,17,10,17,2,30,19,0,1,0,13,9
2,chra,167,10,10,8,14,15,16,2,15,14,2,6,0
3,chra,249,19,1,10,16,8,0,1,0,9,0,18,8
4,chra,288,10,9,13,0,0,20,7,15,8,0,8,12
5,chra,329,13,15,0,1,3,11,4,18,18,4,6,12
6,chra,633,5,7,5,12,15,3,14,4,10,1,8,1


## Run DESeq2

In [42]:
# Run DESeq2 on the "A" counts

dds = DESeqDataSetFromMatrix(countData=counts.A,colData=coldata.A, design = ~ group)
dds$group = factor(dds$group, levels=c('north','south'))
dds = DESeq(dds)

res = results(dds,alpha=0.1)
summary(res)
results.A = cbind(geneloci, as.data.frame(res))


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




out of 1100 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 0, 0%
LFC < 0 (down)     : 1, 0.091%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [43]:
# Run DESeq2 on the "B" counts

dds = DESeqDataSetFromMatrix(countData=counts.B,colData=coldata.B, design = ~ group)
dds$group = factor(dds$group, levels=c('north','south'))
dds = DESeq(dds)

res = results(dds,alpha=0.1)
summary(res)
results.B = cbind(geneloci, as.data.frame(res))


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




out of 1100 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 0, 0%
LFC < 0 (down)     : 0, 0%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



## Merge results tables
Then select for "significant" loci from both A and B. We use a P value cutoff of 0.1. This is the uncorrected P value.

In [54]:
# Merge two results tables
tab.A = results.A[c('Chromosome','Position','pvalue')]
tab.B = results.B[c('Chromosome','Position','pvalue')]
merged = cbind(tab.A, pvalue.B=tab.B$pvalue)

# Select relevant loci from both tables.
subset(merged,pvalue<0.1 & pvalue.B<0.1)

,Chromosome,Position,pvalue,pvalue.B
,<chr>,<int>,<dbl>,<dbl>
201,chrc,42,0.046706966,0.002886657
214,chrc,1029,0.028812214,0.063841347
501,chrf,254,0.009805019,0.020734684
631,chrg,3441,0.014457826,0.019694698
763,chrh,6898,0.015956208,0.081575779
795,chrh,9716,0.040778296,0.048044750
876,chri,7617,0.094598080,0.083604092
1026,chrk,2389,0.076673461,0.048714140


## Conclusion
No genes (loci) have a significant P value according to DESeq2, but we can still use the results to highlight the loci with the biggest changes between the north/south populations. These 8 loci have differences in both A and B modifications.